In [1]:
import fasttext

In [2]:
ft = fasttext.load_model('../cc.en.300.bin')

In [3]:
ft.get_sentence_vector("My name is jeff")[:10]

array([-3.5594115e-03,  1.8136576e-05,  3.6150604e-03,  1.9668644e-02,
       -3.0773604e-02, -3.7558349e-03,  1.1250984e-02,  1.9613564e-02,
        3.4211092e-03,  2.0688642e-03], dtype=float32)

In [4]:
# create a id vector
import json 

f = open("/Users/edgarmonis/Desktop/code/WHO/chitchat-fasttext/data_gen/emoji_data_qna.json",'r')
data = json.load(f)
f.close()

In [5]:
import re
id_data_question = {}
id_data_answer = {}

idx = 0
for x in data.keys():
    id_data_answer[idx]=data[x]
    id_data_question[idx]=re.sub('[^A-Za-z0-9\s]+', '', x)
    idx+=1

In [6]:
vec_labels = {}
for id_ in id_data_question.keys():
    vec_labels[id_] = (id_,ft.get_sentence_vector(id_data_question[id_]))
    # break

In [7]:
import numpy as np

a = np.empty([0,300])

for x in vec_labels.keys():
    vec = vec_labels[x][1]
    a = np.concatenate((a,np.expand_dims(vec,axis=0)),axis=0)

In [8]:
test = a[:10]
test = np.float32(test)
test_id = np.arange(test.shape[0])

print(test.shape)
test_id

test.dtype

(10, 300)


dtype('float32')

In [9]:
# # vec_labels[x]
# import faiss

# index1 = faiss.IndexIDMap2(faiss.IndexFlatL2(300))

# index1.add_with_ids(test,test_id)                  # add vectors to the index

# vec_labels[x]
import faiss

index = faiss.IndexIDMap2(faiss.IndexFlatL2(300))

index.add_with_ids(np.float32(a),np.arange(a.shape[0]))                  # add vectors to the index

In [10]:
# test[:1].shape
D, I = index.search(test[:1], 4)
print(D)
print(I)

[[0.         0.24871458 0.24985558 0.25136146]]
[[   0 9124  553 4423]]


In [11]:
faiss.write_index(index, "./chitchat_emoji_faq.bin")

In [12]:
index2 = faiss.read_index("./chitchat_emoji_faq.bin")
D, I = index2.search(test[:1], 4)

print(D)
print(I)

[[0.         0.24871458 0.24985558 0.25136146]]
[[   0 9124  553 4423]]


In [13]:
json_file_name = "id_emoji_chitchat_answer.json"
print("writing", json_file_name)
with open(json_file_name , 'w') as json_file:
    json.dump(id_data_answer, json_file,\
        indent = 4, sort_keys=True)

json_file_name = "id_emoji_chitchat_question.json"
print("writing", json_file_name)
with open(json_file_name , 'w') as json_file:
    json.dump(id_data_question, json_file,\
        indent = 4, sort_keys=True)

# chitchat_vecs = np.float32(chitchat_vecs)
# chitchat_ids = chitchat_ids.astype(int)

writing id_emoji_chitchat_answer.json
writing id_emoji_chitchat_question.json


In [14]:
import faiss

index = faiss.IndexIDMap2(faiss.IndexFlatL2(300))

index.add_with_ids(chitchat_vecs,chitchat_ids)

NameError: name 'chitchat_vecs' is not defined

In [104]:
faiss.write_index(index, "./chitchat.bin")

In [29]:
vec = np.expand_dims(np.float32(
            ft.get_sentence_vector(
                re.sub('[^A-Za-z0-9\s]+', '',"how are you ?")
            )
        ),axis=0)
D,I = index2.search(vec,1)
id_data_answer[I[0][0]]

'Great, thanks.'

"I'm happy to hear that."